<a href="https://colab.research.google.com/github/Tianarandr/deeplearning-base/blob/main/Cours5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import csv
import keras
import tensorflow as tf
import numpy as np
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Input,LSTM, Dense, Flatten, Dropout, Concatenate, Conv2D, Activation, BatchNormalization, MaxPooling2D
from keras.models import Sequential
from keras.utils import np_utils
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from tensorflow.keras.optimizers import SGD, Adadelta
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

X_test = pd.read_csv('./drive/MyDrive/UCIHARDataset/test/X_test.csv', delimiter='\s+')
y_test = pd.read_csv('./drive/MyDrive/UCIHARDataset/test/y_test.csv', delimiter='\s+')
X_train = pd.read_csv('./drive/MyDrive/UCIHARDataset/train/X_train.csv', delimiter='\s+')
y_train = pd.read_csv('./drive/MyDrive/UCIHARDataset/train/y_train.csv', delimiter='\s+')


X_validation, X_test, y_validation, y_test = train_test_split(X_train, y_train, test_size= 0.5, train_size=0.5, random_state=True)

print("X train : ", X_train.shape)
print("X test: ", X_test.shape)
print("X valisation: ", X_validation.shape)


X train :  (7351, 561)
X test:  (3676, 561)
X valisation:  (3675, 561)


In [ ]:
#normalization 

y_train = np_utils.to_categorical(y_train, 7)
y_test = np_utils.to_categorical(y_test, 7)
y_validation = np_utils.to_categorical(y_validation, 7)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_validation = X_validation.astype('float32')


In [ ]:
def sliding_window(a, window):
    shape = a.shape[:-1]+ (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [ ]:

X_test = np.asarray(X_test)
X_train = np.asarray(X_train)
X_validation = np.asarray(X_validation)



scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train = sliding_window(X_train, 10)
X_validation = sliding_window(X_validation, 10)


In [ ]:
sequence_length = X_train.shape[1] #mety atao 12, hyper paramettre
num_epochs = 3000
batch_size = 1024
print(X_train.shape)
input_dim = X_train.shape[2]
output_dim = y_train.shape[1]


model = Sequential()

model.add(LSTM(64,return_sequences=True, input_shape=(sequence_length, input_dim)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(LSTM(64,return_sequences=True, input_shape=(sequence_length, input_dim)))
model.add(BatchNormalization())
model.add(Dropout(0.25))


model.add(LSTM(128,return_sequences=False, input_shape=(sequence_length, input_dim)))
#model.add(BatchNormalization())
model.add(Dropout(0.25))

#model.add(Dense(1))
model.add(Dense(output_dim))
#model.add(Dense(10, activation="softmax"))
model.add(Activation('softmax'))

checkpointer = ModelCheckpoint(filepath='NN_TP1.hdf5', monitor='val_loss', verbose=0, save_best_only=True, 
                               save_weights_only=False, mode='min')

early = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')


model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size,epochs=num_epochs, validation_data=(X_validation, y_validation), callbacks=[checkpointer, early])






(7351, 552, 10)
Epoch 1/3000
1/8 [==>...........................] - ETA: 3:31 - loss: 2.0322 - accuracy: 0.2393

KeyboardInterrupt: ignored